<a href="https://colab.research.google.com/github/profteachkids/CHE4071_Fall2023/blob/main/TransferFunctionAlgebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
import numpy as np
from plotly.subplots import make_subplots
import scipy as sp
from numpy.polynomial.polynomial import Polynomial as P
from scipy.signal import TransferFunction as TF

In [170]:
def tf2poly(tf):
    return P(tf.num[::-1]), P(tf.den[::-1])

def tf_mult(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*n2).coef[::-1], (d1*d2).coef[::-1])

def tf_div(tf1,tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*d2).coef[::-1], (d1*n2).coef[::-1])

def tf_add(tf1, tf2):
    n1, d1 = tf2poly(tf1)
    n2, d2 = tf2poly(tf2)

    return TF((n1*d2 + n2*d1).coef[::-1], (d1*d2).coef[::-1])

def tf_feedback(tf_f, tf_b):
    nf, df = tf2poly(tf_f)
    nb, db = tf2poly(tf_b)
    return TF((nf*db).coef[::-1], (df*db + nf*nb).coef[::-1])

def pid(Kc, taui, taud):
    return TF([Kc*taud*taui, Kc*taui, Kc], [taui, 0])

In [171]:
tf_process = tf_mult(TF([1.],[2.,1.]), TF([1.],[2.,1.]))

In [172]:
Kc=1
taui=0.7
taud=0.
tf_closed_loop = tf_feedback(tf_mult(pid(Kc, taui, taud), tf_process), TF([1.],[1.]))

/usr/local/lib/python3.10/dist-packages/scipy/signal/_filter_design.py:1746: BadCoefficients:

Badly conditioned filter coefficients (numerator): the results may be meaningless



In [173]:
tf_closed_loop

TransferFunctionContinuous(
array([0.25      , 0.35714286]),
array([1.        , 1.        , 0.5       , 0.35714286]),
dt: None
)

In [174]:
t=np.linspace(0,50,500)
x,y=tf_closed_loop.step(T=t)
fig=make_subplots()
fig.add_scatter(x=x,y=y)
fig.update_layout(width=800,height=600,template='plotly_dark')